# Engine Validation
In this notebook, hypothetical chemical systems are designed specifically to probe aspects of the engine's performance.

In [6]:
from src.simulation_engine import GeneralChemicalSystem, Reaction, ChemicalSpecies
import numpy as np

## Test 1: Mass Positivity and Reactant Depletion
* **Hypothesis:** The solver must not violate the laws of physics by creating negative mass. In a simple irreversible reaction, reactants should be smoothly consumed and approach zero without ever becoming negative.
* **Methdology:**
    1. Define a simple system: `A -> B` (e.g., `Species A`, `Species B`, `Reaction({'A': 1}, {'B': 1}, A=1.0, Ea=0)`).
    2. Set initial conditions: 1 mole of A, 0 moles of B.
    3. Run the simulation to completion.
* **Success Criteria:**
    1. The minimum value of the moles array for both species A and B must be greater than or equal to `0.0` (allowing for a small numerical tolerance, e.g., `-1e-12`).
    2. The final concentration of species A must be less than a small tolerance (e.g., `< 1e-7`), proving the reaction ran to completion.

In [7]:
def test_mass_positivity_and_reactant_depletion():
    print("--- Running Test 1: Mass Positivity and Reactant Depletion ---")
    
    # define a simple system A -> B...
    species_A = ChemicalSpecies('A')
    species_B = ChemicalSpecies('B')
    # use an irreversible reaction with a simple rate...
    reaction = Reaction(reactants={'A': 1}, products={'B': 1}, A=1.0, Ea=0)

    # set initial conditions: 1 mole of A, 0 moles of B...
    system = GeneralChemicalSystem(
        species_list=[species_A, species_B],
        reaction_list=[reaction],
        initial_moles={'A': 1.0, 'B': 0.0},
        initial_V=1.0,
        initial_T=298
    )

    # run the simulation to completion...
    system.run_simulation(rate_tolerance=1e-8, max_iterations=20)

    # --- Success Criterion Checks ---
    # check that moles never went negative during the simulation
    min_moles_A = np.min(system.results['species_data']['A'])
    min_moles_B = np.min(system.results['species_data']['B'])
    
    assert min_moles_A >= -1e-12, f"Mass positivity failed for A. Min moles: {min_moles_A}"
    assert min_moles_B >= -1e-12, f"Mass positivity failed for B. Min moles: {min_moles_B}"
    print("  [PASS] Moles remained positive throughout the simulation.")

    # check that the reactant was consumed
    final_moles_A = system.results['final_moles']['A']
    assert final_moles_A < 1e-7, f"Reactant depletion failed. Final moles of A: {final_moles_A}"
    print(f"  [PASS] Reactant A was successfully depleted to {final_moles_A:.2e} moles.")
    print("--- Test 1 PASSED ---\n")

## Test 2: Conservation of Atoms
* **Hypothesis:** The total number of atoms of each element in the system must remain constant at all times, regardless of the chemical transformations occurring.
* **Methodology:**
    1. Define a system with a clear atomic stoichiometry, for example, a hypothetical dimerization: `2A -> A2`.
    2. Define `Species A` and `Species A2`.
    3. Set initial conditions: 2 moles of A, 0 moles of A2.
    4. Run the simulation. At every time step, calculate the total number of "A" atoms in the system using the formula: `Total A atoms = (1 * moles of A) + (2 * moles of A2)`.
* **Success Criteria:** The calculated Total A atoms at the end of the simulation must be equal to the initial Total A atoms within a very high precision (e.g., relative error < 1e-9).

In [8]:
def test_conservation_of_atoms():
    print("--- Running Test 2: Conservation of Atoms ---")

    # define a system with a dimerization reaction 2A -> A2...
    species_A = ChemicalSpecies('A')
    species_A2 = ChemicalSpecies('A2')
    # a reversible reaction:
    reaction_fwd = Reaction(reactants={'A': 2}, products={'A2': 1}, A=0.8, Ea=500)
    reaction_rev = Reaction(reactants={'A2': 1}, products={'A': 2}, A=0.2, Ea=1000)

    # set initial conditions: 2 moles of A, 0 moles of A2...
    system = GeneralChemicalSystem(
        species_list=[species_A, species_A2],
        reaction_list=[reaction_fwd, reaction_rev],
        initial_moles={'A': 2.0, 'A2': 0.0},
        initial_V=1.0,
        initial_T=300
    )

    # calculate the initial number of 'A' atoms
    initial_A_atoms = (1 * system.initial_moles['A']) + (2 * system.initial_moles.get('A2', 0))

    # run the simulation
    system.run_simulation(rate_tolerance=1e-8)

    # --- Success Criterion Check ---
    # at every time step, calculate the total number of "A" atoms
    moles_A = system.results['species_data']['A']
    moles_A2 = system.results['species_data']['A2']
    
    total_A_atoms_at_each_step = (1 * moles_A) + (2 * moles_A2)
    
    # calculate the relative error at each step
    relative_error = np.abs(total_A_atoms_at_each_step - initial_A_atoms) / initial_A_atoms
    max_relative_error = np.max(relative_error)

    assert max_relative_error < 1e-9, f"Atom conservation failed. Max relative error: {max_relative_error}"
    print(f"  [PASS] Atom count remained constant (Max relative error: {max_relative_error:.2e}).")
    print("--- Test 2 PASSED ---\n")

## Test 3: Thermodynamic Consistency (Equilibrium State)
* **Hypothesis:**  For a reversible reaction, the simulation must converge to the correct, thermodynamically-defined equilibrium state.
* **Methodology:**
    1. Define a reversible system: `A <=> B`. This requires *two* `Reaction` objects:  
        - `r_fwd = Reaction({'A': 1}, {'B': 1}, A=k_f, Ea=0)`
        - `r_rev = Reaction({'B': 1}, {'A': 1}, A=k_r, Ea=0)`
    2. Choose simple rate constants, e.g., `k_f = 0.5`, `k_r = 0.1`.
    3. Calculate the theoretical equilibrium constant: `K_c_theory = k_f / k_r = 0.5 / 0.1 = 5.0`.
    4. Set initial conditions: 1 mole of A, 0 moles of B.
    5. Run the simulation to equilibrium.
    6. From the simulation's final state, calculate the experimental reaction quotient: `Q_c_sim = final_moles_B / final_moles_A`.
* **Success Criteria:** The simulated `Q_c_sim` must match the theoretical `K_c_theory` with less than 0.1% relative error.

In [9]:
def test_thermodynamic_consistency():
    print("--- Running Test 3: Thermodynamic Consistency ---")
    
    # define a reversible system A <=> B
    species_A = ChemicalSpecies('A')
    species_B = ChemicalSpecies('B')
    
    k_f = 0.5
    k_r = 0.1
    
    reaction_fwd = Reaction(reactants={'A': 1}, products={'B': 1}, A=k_f, Ea=0)
    reaction_rev = Reaction(reactants={'B': 1}, products={'A': 1}, A=k_r, Ea=0)

    # calculate the theoretical equilibrium constant
    K_c_theory = k_f / k_r
    print(f"  Theoretical Kc (kf/kr) = {K_c_theory}")

    # set up and run the simulation
    system = GeneralChemicalSystem(
        species_list=[species_A, species_B],
        reaction_list=[reaction_fwd, reaction_rev],
        initial_moles={'A': 1.0, 'B': 0.0},
        initial_V=1.0, # using V = 1.0 dm^3 simplifies moles to concentration
        initial_T=298
    )
    system.run_simulation(rate_tolerance=1e-9)

    # calculate the experimental reaction quotient from the final state
    final_moles = system.results['final_moles']
    final_moles_A = final_moles['A']
    final_moles_B = final_moles['B']
    
    # since V = 1, [X] = moles of X
    Q_c_sim = final_moles_B / final_moles_A
    print(f"  Simulated Qc at equilibrium = {Q_c_sim:.4f}")

    # check if simulated Qc matches theoretical Kc
    relative_error = abs(Q_c_sim - K_c_theory) / K_c_theory
    print(f"  Relative Error = {relative_error:.3%}")
    
    assert relative_error < 0.001, f"Equilibrium constant mismatch. Error: {relative_error:.3%}"
    print("  [PASS] Simulated equilibrium state matches thermodynamic prediction.")
    print("--- Test 3 PASSED ---\n")

## Test 4: Kinetic Consistency (Arrhenius Temperature Dependence)
* **Hypothesis:** The engine must correctly model the effect of temperature on reaction rates as described by the Arrhenius equation.
* **Methodology:**
    1. Define a simple irreversible reaction: `A -> B` with a non-zero activation energy (e.g., `Ea = 50000 J/mol`).
    2. *Run 1:* Simulate the system at a low temperature, `T1 = 300 K`. Record the initial rate of reaction.
    3. *Run 2:* Simulate the exact same system at a higher temperature, `T2 = 320 K`. Record the initial rate of reaction.
* **Success Criteria:** The rate of reaction at `T2` must be significantly greater than the rate at `T1`. For a more rigorous check, the ratio of the simulated rates (`rate_2 / rate_1`) must match the ratio predicted by the Arrhenius equation (`k(T2) / k(T1)`).

In [10]:
def test_kinetic_consistency_arrhenius():
    print("--- Running Test 4: Kinetic Consistency (Arrhenius) ---")

    # define A -> B with non-zero activation energy
    Ea = 50000  # J/mol
    species_A = ChemicalSpecies('A')
    species_B = ChemicalSpecies('B')
    reaction = Reaction(reactants={'A': 1}, products={'B': 1}, A=1e10, Ea=Ea)
    
    initial_moles = {'A': 1.0, 'B': 0.0}
    initial_V = 1.0

    # simulate at low temperature (T1)
    T1 = 300.0  # Kelvin
    system_T1 = GeneralChemicalSystem([species_A, species_B], [reaction], initial_moles, initial_V, T1)
    # we only need the initial rate, so we call get_net_rates directly
    initial_rates_T1 = system_T1.get_net_rates(initial_moles, initial_V, T1)
    rate_at_T1 = abs(initial_rates_T1['A']) # Rate of consumption of A
    print(f"  Initial rate at {T1} K = {rate_at_T1:.4e} mol/s")

    # Run 2: simulate at high temperature (T2)
    T2 = 320.0  # Kelvin
    system_T2 = GeneralChemicalSystem([species_A, species_B], [reaction], initial_moles, initial_V, T2)
    initial_rates_T2 = system_T2.get_net_rates(initial_moles, initial_V, T2)
    rate_at_T2 = abs(initial_rates_T2['A'])
    print(f"  Initial rate at {T2} K = {rate_at_T2:.4e} mol/s")

    # --- Success Criterion ---
    # first, a simple check: rate must increase with temperature for Ea > 0
    assert rate_at_T2 > rate_at_T1, "Rate at T2 should be greater than rate at T1."
    print("  [PASS] Rate correctly increased with temperature.")

    # second, a rigorous check: the ratio of rates must match the Arrhenius prediction
    simulated_ratio = rate_at_T2 / rate_at_T1
    
    # the rate is proportional to the rate constant k, so rate(T2)/rate(T1) should equal k(T2)/k(T1).
    k_T1 = reaction.get_rate_constant(T1)
    k_T2 = reaction.get_rate_constant(T2)
    theoretical_ratio = k_T2 / k_T1
    
    print(f"  Simulated Rate Ratio (rate_2/rate_1) = {simulated_ratio:.4f}")
    print(f"  Theoretical Rate Ratio (k_2/k_1)   = {theoretical_ratio:.4f}")

    # use np.isclose for safe floating-point comparison
    assert np.isclose(simulated_ratio, theoretical_ratio), "Ratio of rates does not match Arrhenius prediction."
    print("  [PASS] Simulated rate change correctly matches Arrhenius equation.")
    print("--- Test 4 PASSED ---\n")

In [12]:
# TO RUN THE TESTS
if __name__ == "__main__":
    print("==============================")
    print("  STARTING SIMULATOR TESTS")
    print("==============================")
    test_mass_positivity_and_reactant_depletion()
    test_conservation_of_atoms()
    test_thermodynamic_consistency()
    test_kinetic_consistency_arrhenius()
    print("==============================")
    print("  ALL TESTS COMPLETED")
    print("==============================")

  STARTING SIMULATOR TESTS
--- Running Test 1: Mass Positivity and Reactant Depletion ---
Starting simulation at T=298K...
Equilibrium reached at t=111.1000s. Max Rate: 3.08e-12
  [PASS] Moles remained positive throughout the simulation.
  [PASS] Reactant A was successfully depleted to 0.00e+00 moles.
--- Test 1 PASSED ---

--- Running Test 2: Conservation of Atoms ---
Starting simulation at T=300K...
Equilibrium reached at t=111.1000s. Max Rate: 1.57e-13
  [PASS] Atom count remained constant (Max relative error: 2.22e-16).
--- Test 2 PASSED ---

--- Running Test 3: Thermodynamic Consistency ---
  Theoretical Kc (kf/kr) = 5.0
Starting simulation at T=298K...
Equilibrium reached at t=111.1000s. Max Rate: 4.92e-13
  Simulated Qc at equilibrium = 5.0000
  Relative Error = 0.000%
  [PASS] Simulated equilibrium state matches thermodynamic prediction.
--- Test 3 PASSED ---

--- Running Test 4: Kinetic Consistency (Arrhenius) ---
  Initial rate at 300.0 K = 1.9675e+01 mol/s
  Initial rate at 